In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sanad-dataset/Finance/1893.txt
/kaggle/input/sanad-dataset/Finance/1711.txt
/kaggle/input/sanad-dataset/Finance/4682.txt
/kaggle/input/sanad-dataset/Finance/5450.txt
/kaggle/input/sanad-dataset/Finance/5064.txt
/kaggle/input/sanad-dataset/Finance/0389.txt
/kaggle/input/sanad-dataset/Finance/3504.txt
/kaggle/input/sanad-dataset/Finance/1773.txt
/kaggle/input/sanad-dataset/Finance/6022.txt
/kaggle/input/sanad-dataset/Finance/0078.txt
/kaggle/input/sanad-dataset/Finance/1812.txt
/kaggle/input/sanad-dataset/Finance/5993.txt
/kaggle/input/sanad-dataset/Finance/1093.txt
/kaggle/input/sanad-dataset/Finance/4417.txt
/kaggle/input/sanad-dataset/Finance/4503.txt
/kaggle/input/sanad-dataset/Finance/1356.txt
/kaggle/input/sanad-dataset/Finance/2869.txt
/kaggle/input/sanad-dataset/Finance/3926.txt
/kaggle/input/sanad-dataset/Finance/2093.txt
/kaggle/input/sanad-dataset/Finance/1880.txt
/kaggle/input/sanad-dataset/Finance/0534.txt
/kaggle/input/sanad-dataset/Finance/3721.txt
/kaggle/in

In [2]:
# Function to load data from folders
def load_data_from_folders(main_folder_path):
    data = []
    for category_folder in os.listdir(main_folder_path):
        category_label = category_folder  # Assuming folder name is the category label
        category_path = os.path.join(main_folder_path, category_folder)
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append({'text': content, 'category': category_label})
    return pd.DataFrame(data)

# Main folder containing subfolders for each category
main_folder_path = '/kaggle/input/sanad-dataset'

# Load data from folders
df = load_data_from_folders(main_folder_path)

In [3]:
df

,text,category
0,"دبي - ""الخليج"":حصدت شعاع كابيتال جائزة ""أفضل ش...",Finance
1,أبوظبي - علي أسعد: تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي «الخليج»: أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية، التي تتخذ من د...,Finance
...,...,...
45495,على هامش مشاركتها في أسبوع جيتكس للتقنية أعلنت...,Tech
45496,أعلنت شركة إيه أم دي، عن طرح أحدث منصات الكمبي...,Tech
45497,"أبوظبي ""الخليج"":زار الفريق سيف عبدالله الشعفار...",Tech
45498,تشارك وزارة الاقتصاد في الدورة ال 33 لأسبوع جي...,Tech


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45500 entries, 0 to 45499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      45500 non-null  object
 1   category  45500 non-null  object
dtypes: object(2)
memory usage: 711.1+ KB


In [5]:
df.describe()

,text,category
count,45500,45500
unique,45485,7
top,إعداد: محمد صالح القرق دخل على رسول الله- صلى ...,Finance
freq,3,6500


In [8]:
df.isnull().sum()

text        0
category    0
dtype: int64

In [9]:
#lowercasing and removing special characters
df['text'] = df['text'].str.lower().str.replace(r'\W', ' ', regex=True)

In [10]:
df

,text,category
0,دبي الخليج حصدت شعاع كابيتال جائزة أفضل ش...,Finance
1,أبوظبي علي أسعد تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي الخليج أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية التي تتخذ من د...,Finance
...,...,...
45495,على هامش مشاركتها في أسبوع جيتكس للتقنية أعلنت...,Tech
45496,أعلنت شركة إيه أم دي عن طرح أحدث منصات الكمبي...,Tech
45497,أبوظبي الخليج زار الفريق سيف عبدالله الشعفار...,Tech
45498,تشارك وزارة الاقتصاد في الدورة ال 33 لأسبوع جي...,Tech


In [12]:
pip install transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import faiss
from transformers import DistilBertTokenizer, DistilBertModel
import torch


In [14]:
# Initialize DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [15]:
def encode_documents(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Take the mean of token embeddings as document embeddings
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)
    return embeddings


In [18]:
# Sample 1,000 rows
df = df.sample(n=1000, random_state=42) 

In [20]:
df.shape

(1000, 2)

In [21]:
# Encode documents
document_embeddings = encode_documents(df['text'].tolist())

In [22]:
# Create FAISS index
dimension = document_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))


In [23]:
# Save the index
faiss.write_index(index, 'document_index.faiss')

In [24]:
from transformers import pipeline

# Load FAISS index
index = faiss.read_index('document_index.faiss')

In [31]:
from transformers import pipeline

# Initialize text generation pipeline
generator = pipeline('text-generation', model='gpt2')

def generate_answer(query):
    # Encode the query
    query_embedding = encode_documents([query])[0]
    
    # Retrieve relevant documents
    _, indices = index.search(np.array([query_embedding]), k=5)  # Retrieve top 5 documents
    retrieved_docs = [df.iloc[idx]['text'] for idx in indices[0]]
    
    # Combine retrieved documents into a context for generation
    context = ' '.join(retrieved_docs)
    
    # Ensure the context is within the acceptable length
    if len(context) > 1024:
        context = context[:1024]  # truncate to fit model's max length

    # Generate response
    generated = generator(f"Context: {context}\n\nQuestion: {query}", max_length=200, max_new_tokens=150)
    
    return generated[0]['generated_text']

# Test the model
print(generate_answer("What is the main theme of the dataset?"))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Context: ارتفع مؤشر الصقر الصادر عن المال كابيتال 43 45 نقطة توازي 2  ليغلق عند 2149 91 نقطة  وكانت أكثر الشركات ارتفاعا   الواحة بنسبة 8 8   الاتحاد العقارية 5 48   أملاك 4 69   دانة غاز 4 48    تصدر جهاز iomega ego portable hard drive 500gb لائحة أفضل خمسة أقراص صلبة خارجية قابلة للحمل والتنقل بحسب موقع بي سي ورلد 1iomega ego portable hard drive 500gb  السعة  500 غيغابايت    المنافذ المدعومة  firewire 400  firewire 800  usb 2  0  الملحقات  usb 2  0   power cable  firewire 400cable  firewire 800 cable   البرامج الملحقة  retrospect express  iomega quikprotect  الضمان  3 سنوات    السعر عند اعداد التقرير  150 دولار  2western digital mypassport essential se 1tb   السعة  1000غيغابايت    المنافذ المدعومة  usb 2  0  سرعة الدوران  5400 دورة في الدقيقة    الملحقات  mini usb cable  البرامج الملحقة  wd smartware   الضمان  سنة واحدة    السعر عند اعداد التقرير  25  178 دولار  3wiebetech toughtechxe mini 500gb   السعة  500 غيغابايت    المنافذ المدعومة  external sata 300  firewire 800  usb 2  0  سرع

In [32]:
# Test the model
print(generate_answer("التهاب الكبد الوبائي"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Context: صادقت الجمعية العمومية لشركة  دبي الوطني للتأمين وإعادة التأمين  المنعقدة أمس على توزيع أرباح نقدية للمساهمين بنسبة 20  من رأس المال عن عام 2013   أعلن سوق دبي المالي عن إيداع أسهم المنحة لشركة بيت التمويل الخليجي بنسبة 20  في حسابات المساهمين وذلك كما في تاريخ إغلاق سجل الشركة في 22 2 2009   أعلن مجلس إدارة شركة سوق دبي المالي أنه اجتمع يوم الاثنين بتاريخ 27 9 2010 واعتمد البنود المتعلقة بتوصيات لجنة الترشيحات والمكافآت  والهيكل التنظيمي للشركة   ينظم بيت الشعر ندوة حوارية بعنوان دغدغة قصيدة النثر  يشترك فيها د   بهجت الحديثي وعبداللطيف الزبيدي ويقدم الندوة ويديرها عبدالفتاح صبري  وذلك في الثامنة من مساء اليوم   ينعقد اجتماع شركة صروح العقارية يوم 27 يوليو تموز الجاري  وذلك لمناقشة الانجازات المالية للربع الثاني من عام  2011 ومراجعة أمور عامة وتشغيلية واستراتيجية  

Question: التهاب الكبد الوبائيركمة 2 لجلى توزيع الوطني التأمين  المنعقدة توزيع آنا من يكان باند إيداع المان رأس الموطني القومدة لمس أمس عرج وقهم مشرقة مسؤول ما ورجد لكان في معتكان ال
